In [ ]:
# import ee

# SERVICE_ACCOUNT = "gee-map-bot@your-project.iam.gserviceaccount.com"
# KEY_FILE = r"C:\Users\semro\OneDrive - Delft University of Technology\Shoreline_change\gee-leaflet-map-57cb3c98a2b8.json"

# credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
# ee.Initialize(credentials)

# print("✅ Earth Engine initialized with Service Account")

import ee, os

SERVICE_ACCOUNT = os.environ.get("GEE_SERVICE_ACCOUNT")
KEY_FILE = "key.json"   # GitHub Actions writes the secret here

credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_FILE)
ee.Initialize(credentials)

print("✅ Earth Engine initialized with Service Account")

import geemap.foliumap as geemap
import folium
import json

# ---------------- Helper functions -----------------

def generate_tile_url(image, vis_params=None):
    """Generate tile URL for a given EE image."""
    map_id_dict = ee.Image(image).getMapId(vis_params or {})
    return map_id_dict["tile_fetcher"].url_format

# ------------------- Your AOI ----------------------
# Replace this with your AOI
# aoi = ee.Geometry.Rectangle([105.0, 8.5, 106.5, 10.0])

# ----------------- Get composites ------------------
def get_landsat_composite(collection, year, aoi):
    return (
        ee.ImageCollection(collection)
        .filterBounds(aoi)
        .filterDate(f"{year}-01-01", f"{year}-12-31")
        .median()
        .clip(aoi)
    )

def get_sentinel_composite(collection, year, aoi):
    return (
        ee.ImageCollection(collection)
        .filterBounds(aoi)
        .filterDate(f"{year}-01-01", f"{year}-12-31")
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .median()
        .clip(aoi)
    )

def add_ndvi(image, sensor):
    if sensor in ["L5", "L7"]:  # Landsat 5 or 7
        ndvi = image.normalizedDifference(["SR_B4", "SR_B3"]).rename("NDVI")
    elif sensor in ["L8", "L9"]:  # Landsat 8 or 9
        ndvi = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    elif sensor == "S2":  # Sentinel-2
        ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    else:               # error als niet bekend 
        raise ValueError("Unknown sensor type")
    return image.addBands(ndvi)

aoi = ee.Geometry.Polygon(
    [
    [104.83407034518385,8.498420952237657],
    [105.29801122715598,8.619546673827465],
    [105.65541439538752,9.022594535155221],
    [106.57553209899334,9.458041731667457],
    [106.87668476825814,10.104561549327723],
    [106.81079328752804,10.446530587537117],
    [106.76416369437943,10.424908044751785],
    [106.77809252379802,10.357293904341999],
    [106.77772583336036,10.29199817498727],
    [106.75300659507911,10.273418700336107],
    [106.70476974815529,10.286762251847419],
    [106.7003065523545,10.274769971702582],
    [106.74247561138289,10.261256998193668],
    [106.74590883892195,10.243689270169133],
    [106.74161730449812,10.228485642816539],
    [106.72513781231062,10.219532054945693],
    [106.69818697612898,10.236087547557192],
    [106.68428240459578,10.212605521460105],
    [106.73859909866802,10.180646123990934],
    [106.77876786087505,10.150739045893204],
    [106.77619294022075,10.126912762691054],
    [106.7661089546316,10.11099217958563],
    [106.70348727313478,10.059379210977996],
    [106.65453728404505,10.026869675624008],
    [106.62895973887903,9.97801359588208],
    [106.57419975963099,10.00032939502244],
    [106.56184014049036,9.984099874718403],
    [106.61690756811764,9.940565201740917],
    [106.66377112402584,9.900997087774805],
    [106.66219354725415,9.879965634308538],
    [106.66373849964673,9.868381016411186],
    [106.6597902879768,9.852906037203724],
    [106.64603540609954,9.845156438264155],
    [106.62577936361907,9.822914648051116],
    [106.60020181845306,9.8147955991504],
    [106.57273599814056,9.847101466592218],
    [106.50655184773105,9.910444020648574],
    [106.49299059895175,9.90165071107393],
    [106.54294405964511,9.850069831816894],
    [106.52800951985019,9.8333254502573],
    [106.51856814411776,9.839414414493177],
    [106.47307787922519,9.890658742849233],
    [106.45488177326816,9.872394352044438],
    [106.46638308552401,9.857173252245838],
    [106.48598644375815,9.838025656918534],
    [106.50134078078969,9.815849616139126],
    [106.51318541579946,9.799695551350213],
    [106.51112646826252,9.7843739052923],
    [106.51653380163654,9.760882883750469],
    [106.52340025671467,9.75445411907771],
    [106.53535365245101,9.74824407429176],
    [106.54513835093734,9.747821117021044],
    [106.55749797007796,9.744352847170891],
    [106.56342565122502,9.745721750067938],
    [106.57381136482394,9.695203009411424],
    [106.57518465583956,9.676081775838208],
    [106.5705050216768,9.653438856132821],
    [106.55031475523022,9.607517591504283],
    [106.51821407773998,9.575357866360834],
    [106.50256769605087,9.58385266187482],
    [106.42403911636447,9.548461292235471],
    [106.40223812149142,9.544229187944048],
    [106.38713192031955,9.536441978734958],
    [106.37065242813205,9.556248226727421],
    [106.3350612273578,9.586942885645778],
    [106.30605045465272,9.598960413627076],
    [106.29092899169142,9.577197411267806],
    [106.26658753861153,9.523765694017928],
    [106.24049500931466,9.50311103379364],
    [106.22744874466622,9.511068304742269],
    [106.21493603059194,9.53981175665193],
    [106.22815395661733,9.571974846972395],
    [106.19193340608022,9.595502759006667],
    [106.17047573396108,9.57840710002702],
    [106.17476726838491,9.549122437406348],
    [106.18120131293784,9.5372086700095],
    [106.20331631753396,9.499089754352273],
    [106.20125638101052,9.478772240688475],
    [106.1948113638213,9.463388315196985],
    [106.17893268645314,9.440443983574037],
    [106.17326786101368,9.440020642056666],
    [106.15790416777638,9.427320154802596],
    [106.14981134491401,9.4152371149228],
    [106.15633447723823,9.407870360406971],
    [106.1707540329023,9.408124389038006],
    [106.17289980011421,9.406684890994804],
    [106.1719542071107,9.38885140968987],
    [106.14089577478988,9.358969879587496],
    [106.1346301345311,9.357276111764355],
    [106.13196938318832,9.351940689198507],
    [106.11234410650688,9.34496325851762],
    [106.05694039709526,9.32789748425577],
    [106.00886069274415,9.311285116581564],
    [105.98568640685548,9.30361970317263],
    [105.97442514477547,9.298697810478862],
    [105.94132479085556,9.288720985474827],
    [105.92789228810898,9.284443345023716],
    [105.89090923650754,9.271929366191602],
    [105.88863472326291,9.27031988597354],
    [105.85687916911566,9.257641523314545],
    [105.83078663981878,9.247348775152632],
    [105.82829754985296,9.245781540167336],
    [105.82065861857855,9.24357892785211],
    [105.81006095915637,9.23945205670955],
    [105.78151835381354,9.226447739431682],
    [105.74993266045416,9.211324852257405],
    [105.74555529534186,9.2110283185837],
    [105.74012109284735,9.208473329748871],
    [105.7174188757453,9.198687508229101],
    [105.69857004190408,9.192036384111324],
    [105.64749261032787,9.173649824325224],
    [105.62461873184887,9.16547300650149],
    [105.60162055444168,9.153322665158798],
    [105.57924879211876,9.141459184216133],
    [105.54547441620323,9.131332543115594],
    [105.53255437318751,9.12391745607673],
    [105.49628435144443,9.103578140859891],
    [105.4810064888956,9.093365673226678],
    [105.47907963249436,9.091524922762042],
    [105.4652179763054,9.07364171713917],
    [105.45901539534371,9.067651188758015],
    [105.45021774977486,9.055530582013587],
    [105.44918778151315,9.055149157683644],
    [105.44517430676736,9.050718484260683],
    [105.42887935316944,9.030238431151563],
    [105.4218841520586,9.021570914302972],
    [105.42033919966602,9.021062300488946],
    [105.4413548299933,9.110793392398033],
    [105.43277176114564,9.09452143524365],
    [105.42387850719459,9.091672587030235],
    [105.41495211559302,9.071500953063323],
    [105.39156208395393,9.066660315101288],
    [105.38522865104096,9.015277580227394],
    [105.38316871451752,9.000696779462878],
    [105.36480094718354,8.979163592091664],
    [105.35038139151948,8.974076825566744],
    [105.32755042838471,8.957289990237898],
    [105.32583381461518,8.940163228021191],
    [105.32583381461518,8.920491893635855],
    [105.31563203913947,8.90181821507526],
    [105.29880922419807,8.871968506010491],
    [105.27280252558967,8.834992206069172],
    [105.26782434565803,8.836518824075963],
    [105.25761049372932,8.832617454381055],
    [105.25400666792818,8.821962266072223],
    [105.25400666792818,8.809579027186784],
    [105.24636773665377,8.805253277464384],
    [105.23944385809962,8.792331871649184],
    [105.22390850348536,8.784952277296],
    [105.21661289496485,8.777657361499642],
    [105.19510257105391,8.767290693527618],
    [105.18497454981367,8.761160212764004],
    [105.17802226404707,8.751065400484277],
    [105.1687525496916,8.74919910267145],
    [105.16265857080977,8.742582153233307],
    [105.1450047267704,8.730397284729255],
    [105.14139983785438,8.723695163872327],
    [105.13564918172645,8.718350348357333],
    [105.12861106527137,8.715635492144727],
    [105.1259503139286,8.710714764999242],
    [105.12243125570106,8.708678583100651],
    [105.11686197756347,8.701048565154228],
    [105.11085382937011,8.699436547092226],
    [105.1083647394043,8.696042802174036],
    [105.10268155620544,8.691793547200271],
    [105.09469930217712,8.6901814893128],
    [105.09092275188415,8.683902881997968],
    [105.0856012491986,8.681866554351451],
    [105.08354131267517,8.676775686891867],
    [105.0742715983197,8.66863015538961],
    [105.06980840251892,8.667611951534356],
    [105.06339797597549,8.65877683098845],
    [105.05472907643936,8.653261382635145],
    [105.04956735233321,8.64511534190111],
    [105.03875268558517,8.637817697538495],
    [105.03720773319259,8.632471659148035],
    [105.02707971195235,8.625428350113435],
    [105.016734921179,8.62287064120337],
    [104.99759467764872,8.61714251330338],
    [104.98656222848398,8.609706144068117],
    [104.95918223885995,8.602874573451276],
    [104.94915314380141,8.59663694482554],
    [104.91413422290297,8.585943628377393],
    [104.89550896350356,8.583482425570688],
    [104.87671166670381,8.57618359239772],
    [104.84469681990205,8.576268463356898],
    [104.80580419262468,8.58858815640435],
    [104.77576345165788,8.596395950025112],
    [104.77353185375749,8.60029978652925],
    [104.76503461559831,8.596395950025112],
    [104.7617730494362,8.599281398275311],
    [104.75602239330827,8.598941934915688],
    [104.75310414990007,8.591982869036848],
    [104.83407034518385,8.498420952237657]  # closing point
])


# ----------------- Create composites -----------------
landsat1984 = add_ndvi(get_landsat_composite("LANDSAT/LT05/C02/T1_L2", 1988, aoi), "L5")
landsat1990 = add_ndvi(get_landsat_composite("LANDSAT/LT05/C02/T1_L2", 1992, aoi), "L5")
landsat1995 = add_ndvi(get_landsat_composite("LANDSAT/LT05/C02/T1_L2", 1997, aoi), "L5")
landsat2000 = add_ndvi(get_landsat_composite("LANDSAT/LE07/C02/T1_L2", 2001, aoi), "L7")
landsat2005 = add_ndvi(get_landsat_composite("LANDSAT/LE07/C02/T1_L2", 2005, aoi), "L7")
landsat2010 = add_ndvi(get_landsat_composite("LANDSAT/LE07/C02/T1_L2", 2010, aoi), "L7")
landsat2015 = add_ndvi(get_landsat_composite("LANDSAT/LC08/C02/T1_L2", 2015, aoi), "L8")
sentinel2020 = add_ndvi(get_sentinel_composite("COPERNICUS/S2_SR_HARMONIZED", 2020, aoi), "S2")
sentinel2025 = add_ndvi(get_sentinel_composite("COPERNICUS/S2_SR_HARMONIZED", 2025, aoi), "S2")

# ----------------- NDVI threshold -------------------
ndvi_threshold = 0.1
mangrove_1984 = landsat1984.select("NDVI").gt(ndvi_threshold)
mangrove_1990 = landsat1990.select("NDVI").gt(ndvi_threshold)
mangrove_1995 = landsat1995.select("NDVI").gt(ndvi_threshold)
mangrove_2000 = landsat2000.select("NDVI").gt(ndvi_threshold)
mangrove_2005 = landsat2005.select("NDVI").gt(ndvi_threshold)
mangrove_2010 = landsat2010.select("NDVI").gt(ndvi_threshold)
mangrove_2015 = landsat2015.select("NDVI").gt(ndvi_threshold)
mangrove_2020 = sentinel2020.select("NDVI").gt(ndvi_threshold)
mangrove_2025 = sentinel2025.select("NDVI").gt(ndvi_threshold)

# ----------------- Loss layers ----------------------
loss_1984_1990 = mangrove_1984.And(mangrove_1990.Not())
loss_1990_1995 = mangrove_1990.And(mangrove_1995.Not())
loss_1995_2000 = mangrove_1995.And(mangrove_2000.Not())
loss_2000_2005 = mangrove_2000.And(mangrove_2005.Not())
loss_2005_2010 = mangrove_2005.And(mangrove_2010.Not())
loss_2010_2015 = mangrove_2010.And(mangrove_2015.Not())
loss_2015_2020 = mangrove_2015.And(mangrove_2020.Not())
loss_2020_2025 = mangrove_2020.And(mangrove_2025.Not())

# ----------------- Map setup ------------------------
Map = geemap.Map(center=[9.2, 105.75], zoom=11)

# Dictionary to store tile URLs
tile_urls = {}

# --------------- Set Colors for layers and legend
color_1984_1990 = "#2C0035"  # Very dark violet-blue (oldest)
color_1990_1995 = "#4D004B"
color_1995_2000 = "#810F7C"
color_2000_2005 = "#8856A7"
color_2005_2010 = "#8C96C6"
color_2010_2015 = "#9EBCDA"
color_2015_2020 = "#BFD3E6"
color_2020_2025 = "#E0F3F8"  # Pale blue (most recent)


# Add loss layers with palette and generate tile URLs
loss_layers = {
    "Mangrove loss (1984-1992)": (loss_1984_1990, color_1984_1990),
    "Mangrove loss (1992-1997)": (loss_1990_1995, color_1990_1995),
    "Mangrove loss (1997-2001)": (loss_1995_2000, color_1995_2000),
    "Mangrove loss (2001-2005)": (loss_2000_2005, color_2000_2005),
    "Mangrove loss (2005-2010)": (loss_2005_2010, color_2005_2010),
    "Mangrove loss (2010-2015)": (loss_2010_2015, color_2010_2015),
    "Mangrove loss (2015-2020)": (loss_2015_2020, color_2015_2020),
    "Mangrove loss (2020-2025)": (loss_2020_2025, color_2020_2025),
}


# Example for the 2024 Sentinel layer
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
           .filterBounds(aoi) \
           .filterDate("2024-01-01", "2024-12-31") \
           .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) \
           .median()

# Add Sentinel-2 as basemap (true color RGB)
Map.addLayer(
    sentinel,
    {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000},
    "Sentinel-2 (2024)"
)
mapid_dict = sentinel.getMapId({'bands': ['B4','B3','B2'], 'min':0, 'max':3000})
tile_urls['Sentinel-2 (2024)'] = mapid_dict['tile_fetcher'].url_format



for name, (layer, color) in loss_layers.items():
    vis = {"palette": [color]}
    Map.addLayer(layer.updateMask(layer), vis, name)
    tile_urls[name] = generate_tile_url(layer, vis)

# Save tile URLs to JSON
with open("tile_urls.json", "w") as f:
    json.dump(tile_urls, f, indent=2)


# --------------Add Legend --------------------
legend_dict = {
    "Mangrove loss (1984-1992)": color_1984_1990,
    "Mangrove loss (1992-1997)": color_1990_1995,
    "Mangrove loss (1997-2001)": color_1995_2000,
    "Mangrove loss (2001-2005)": color_2000_2005,
    "Mangrove loss (2005-2010)": color_2005_2010,
    "Mangrove loss (2010-2015)": color_2010_2015,
    "Mangrove loss (2015-2020)": color_2015_2020,
    "Mangrove loss (2020-2025)": color_2020_2025
    }

style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}
def add_folium_legend(m, title, legend_dict, style=None):
    """Add a custom legend to a folium map."""
    if style is None:
        style = {
            "position": "fixed",
            "z-index": "9999",
            "border": "2px solid grey",
            "background-color": "rgba(255, 255, 255, 0.8)",
            "border-radius": "10px",
            "padding": "5px",
            "font-size": "14px",
            "bottom": "20px",
            "right": "5px",
        }

    style_str = ";".join([f"{k}:{v}" for k, v in style.items()])
    legend_html = f'<div style="{style_str}">'
    legend_html += f"<b>{title}</b><br>"

    for label, color in legend_dict.items():
        legend_html += (
            f'<i style="background:{color};width:15px;height:15px;'
            f'display:inline-block;margin-right:5px;"></i>{label}<br>'
        )

    legend_html += "</div>"
    m.get_root().html.add_child(folium.Element(legend_html))

add_folium_legend(Map, "Mangrove Loss in the <br> Mekong Delta (1984-2025)", legend_dict, style=style)

# ------------- Add scalebar  ----------------------------

from branca.element import MacroElement, Element
from jinja2 import Template

class ScaleBar(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
        L.control.scale({
            position: 'bottomleft',
            metric: true,
            imperial: false,
            maxWidth: 300
        }).addTo({{this._parent.get_name()}});
        {% endmacro %}
    """)

    def __init__(self, position="bottomleft", metric=True, imperial=False, max_width=300, font_size="16px"):
        super().__init__()
        self._name = "ScaleBar"
        self.position = position
        self.metric = metric
        self.imperial = imperial
        self.max_width = max_width
        self.font_size = font_size

    def render(self, **kwargs):
        super().render(**kwargs)
        # Inject CSS for larger font
        css = f"""
        <style>
        .leaflet-control-scale-line {{
            font-size: {self.font_size} !important;
            font-weight: bold;
        }}
        </style>
        """
        self.get_root().html.add_child(Element(css))

Map.add_child(ScaleBar(font_size="14px"))


# ----------- north arrow ----------------
def add_north_arrow(m, position="topright", arrow_size="35px", text_size="25px"):
    arrow_css = f"""
        <div style="
            position: absolute; 
            { 'top: 10px; right: 10px;' if position=='topright' else '' }
            { 'top: 10px; left: 10px;' if position=='topleft' else '' }
            { 'bottom: 10px; right: 10px;' if position=='bottomright' else '' }
            { 'bottom: 30px; left: 10px;' if position=='bottomleft' else '' }
            z-index: 9999; 
            font-weight: bold; 
            color: black;
            text-shadow: 1px 1px 2px white;
            display: flex;
            align-items: center;
        ">
            <span style="font-size:{arrow_size}; line-height:1;">↑</span>
            <span style="font-size:{text_size}; margin-left: 4px; line-height:1;">N</span>
        </div>
    """
    m.get_root().html.add_child(folium.Element(arrow_css))


add_north_arrow(Map, position="bottomleft", arrow_size="35px", text_size="25px")




# --------------- Save as standalone HTML --------------
Map.to_html("tryout_goal_LOSS_exp.html")
print("Map saved as HTML and tile URLs saved to tile_urls.json")


Map saved as HTML and tile URLs saved to tile_urls.json
